In [ ]:
sd = SuferData( freesurfer_data_folder = "/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/",
                path_base_mri = "/home/brunovieira/Tamires_Experiments/Bases_de_Dados/")
sd.monitor()
sd.save_files()